In [42]:
import pandas as pd
import numpy as np
import time

np.random.seed(33)
df = pd.read_csv("../data/featureSelectedAllDataWithY.csv")
# df = pd.read_csv("../data/Cleaned_dat_encoded.csv")
print(df.shape)
df.head()

(534730, 21)


,esi,age,ethnicity,race,lang,maritalstatus,employstatus,insurance_status,arrivalmode,previousdispo,...,meds_analgesics,meds_antiplateletdrugs,meds_cardiacdrugs,meds_cardiovascular,meds_diuretics,meds_elect/caloric/h2o,meds_gastrointestinal,meds_psychotherapeuticdrugs,meds_vitamins,disposition
0,4,40,1,8,1,8,2,4,6,7,...,0,0,0,0,0,0,0,0,0,2
1,4,66,1,4,1,5,3,1,2,7,...,0,0,0,0,0,0,0,0,0,2
2,2,66,1,4,1,5,3,1,6,3,...,0,0,0,0,0,0,0,0,0,2
3,2,66,1,4,1,5,3,1,2,3,...,0,0,0,0,0,0,0,0,0,2
4,3,84,1,5,2,10,6,3,6,3,...,0,0,0,2,1,2,2,0,1,1


## Correlation analysis

In [43]:
health_data = df.copy()

feature_corr = health_data.corr()
feature_corr[abs(feature_corr['disposition']) > 0.1]['disposition']

esi                            0.431877
age                           -0.374109
race                          -0.136434
employstatus                  -0.135497
arrivalmode                    0.177370
previousdispo                  0.145210
htn                           -0.261439
meds_analgesics               -0.338007
meds_antiplateletdrugs        -0.333516
meds_cardiacdrugs             -0.308993
meds_cardiovascular           -0.441590
meds_diuretics                -0.317732
meds_elect/caloric/h2o        -0.308729
meds_gastrointestinal         -0.390537
meds_psychotherapeuticdrugs   -0.323160
meds_vitamins                 -0.348034
disposition                    1.000000
Name: disposition, dtype: float64

In [44]:
print(feature_corr[abs(feature_corr['disposition']) > 0.2]['disposition'])
print(len(feature_corr[abs(feature_corr['disposition']) > 0.2]['disposition']))


esi                            0.431877
age                           -0.374109
htn                           -0.261439
meds_analgesics               -0.338007
meds_antiplateletdrugs        -0.333516
meds_cardiacdrugs             -0.308993
meds_cardiovascular           -0.441590
meds_diuretics                -0.317732
meds_elect/caloric/h2o        -0.308729
meds_gastrointestinal         -0.390537
meds_psychotherapeuticdrugs   -0.323160
meds_vitamins                 -0.348034
disposition                    1.000000
Name: disposition, dtype: float64
13
